In [1]:
from ipywidgets import interact
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time, math

In [7]:
from sequence.kernel.timeline import Timeline
from sequence.topology.node import Node
from sequence.components.light_source import SPDCSource
from sequence.kernel.process import Process
from sequence.kernel.event import Event
from sequence.utils import log
import logging
logger = logging.getLogger()
from sequence.kernel.timeline import Timeline
from sequence.protocol import Protocol
#logger.setLevel(logging.DEBUG)

FREQUENCY = 1e5
#Qbits_NUM = 2


class Counter:
    def __init__(self):
        self.count = 0
        self.time = 0

    def trigger(self, detector, info):
        self.count += 1
        self.time = info['time']
        
        
# protocol to control photon emission on end node
class EmitProtocol(Protocol):
    def __init__(self, own: "EndNode", name: str, num_qubits: int, source_name: str):
        """Constructor for Emission protocol.

        Args:
            own (EndNode): node on which the protocol is located.
            name (str): name of the protocol instance.
            num_qubits (int): number of qubits to send in one execution.
            delay_time (int): time to wait before re-starting execution.
            source_name (str): name of the light source on the node.
        """

        super().__init__(own, name)
        self.num_qubits = num_qubits
        self.source_name = source_name
        
    def start(self):
        states = [[complex(math.sqrt(1/2)), complex(math.sqrt(1/2))]] * self.num_qubits  # TODO: rewrite spdc class?
        source = self.own.components[self.source_name]
        source.emit(states)

    def received_message(self, src: str, msg):
        pass
        



class SenderNode(Node):
    def __init__(self, name, timeline, Qbits_NUM):
        super().__init__(name, timeline)
        self.n_pairs = 0
        self.timeline = timeline
        
        SPDCSource_name = name + ".spdcsource"
        self.spdcsource = SPDCSource(SPDCSource_name, timeline, frequency=FREQUENCY, mean_photon_num=5,random_frequency=True)
        
        self.add_component(self.spdcsource)
        
        self.spdcsource.add_receiver(self)
        self.spdcsource.add_receiver(self)
        self.periods = []
        self.counts = []
        self.times = []
        process = Process(self, "init_counter",[])
        period = int(1e12 / FREQUENCY)
        for i in range(Qbits_NUM):
            event = Event(i * period + period, process)
            self.periods.append(i * period)
            self.timeline.schedule(event)
        
        
        
        # protocols
        self.emit_protocol = EmitProtocol(self, name + ".emit_protocol", Qbits_NUM, SPDCSource_name)

    def init_counter(self):
        self.counts.append(self.n_pairs//2)
        self.n_pairs = 0



    def get(self, photon: "Photon", **kwargs):
        self.n_pairs +=1
        self.times.append(self.timeline.time)


        
def test(sim_time,mean_photon_num,Qbits_NUM):
    PS_PER_MS = 1e9
    tl = Timeline(sim_time * PS_PER_MS)
    my_source = SenderNode("my_source",tl, Qbits_NUM)
    my_source.set_seed(0)
    my_source.spdcsource.mean_photon_num = mean_photon_num
    tl.init()
    process = Process(my_source.emit_protocol, "start", [])
    event = Event(0, process)
    tl.schedule(event)
    tl.run()
    
    plt.figure()
    
    x=my_source.times
    y=[1]*len(x)
    
    counts = my_source.spdcsource.num_photon_pairs_list
    #counts = my_source.counts
    plt.plot(x,y, marker ='x')
    #plt.plot(data,range(1, len(data) + 1),marker ='x')
    df = pd.DataFrame({"counts":counts})
    
    data_count = df[["counts"]].apply(pd.value_counts)
    
    
    data_count.sort_index().plot(kind="bar")
    plt.show()

In [8]:
interactive_plot = interact(test, sim_time=(100, 1000, 100),mean_photon_num=(0, 50, 1), Qbits_NUM=(0, 1000, 1))
interactive_plot

interactive(children=(IntSlider(value=500, description='sim_time', max=1000, min=100, step=100), IntSlider(val…

<function __main__.test(sim_time, mean_photon_num, Qbits_NUM)>